# Implementing an LLM-powered recommendation system

In [ ]:
import os
os.chdir('/desired/default/directory/path')


In [1]:
pwd

'/Users/leiyu/Projects/llm/notebooks'

## Data Preprocessing

In [1]:
import pandas as pd

md = pd.read_csv('./data/movies_metadata.csv')
md.head()

/var/folders/lj/sq_z3m_s3z51_g2s4qfv2hrw0000gn/T/ipykernel_12906/2682188723.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('./data/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [2]:
import pandas as pd
import ast


# Convert string representation of dictionaries to actual dictionaries
md['genres'] = md['genres'].apply(ast.literal_eval)

# Transforming the 'genres' column
md['genres'] = md['genres'].apply(lambda x: [genre['name'] for genre in x])

md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# Calculate weighted rate (IMDb formula)
def calculate_weighted_rate(vote_average, vote_count, min_vote_count=10):
    return (vote_count / (vote_count + min_vote_count)) * vote_average + (min_vote_count / (vote_count + min_vote_count)) * 5.0

# Minimum vote count to prevent skewed results
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
min_vote_count = vote_counts.quantile(0.95)

# Create a new column 'weighted_rate'
md['weighted_rate'] = md.apply(lambda row: calculate_weighted_rate(row['vote_average'], row['vote_count'], min_vote_count), axis=1)
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,weighted_rate
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,7.499658
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,6.610362
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,5.262357
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,5.079915
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5.199506


In [4]:
md = md.dropna()

In [5]:
md_final = md[['genres', 'title', 'overview', 'weighted_rate']].reset_index(drop=True)
md_final.head()

,genres,title,overview,weighted_rate
0,"[Adventure, Action, Thriller]",GoldenEye,James Bond must unmask the mysterious head of ...,6.173464
1,[Comedy],Friday,Craig and Smokey are two guys in Los Angeles h...,6.083421
2,"[Horror, Action, Thriller, Crime]",From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,6.503176
3,[Comedy],Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",5.109091
4,"[Action, Adventure, Science Fiction, Family, F...",Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,5.052129


In [6]:
# Create a new column by combining 'title', 'overview', and 'genre'
md_final['combined_info'] = md_final.apply(lambda row: f"Title: {row['title']}. Overview: {row['overview']} Genres: {', '.join(row['genres'])}. Rating: {row['weighted_rate']}", axis=1)
md_final['combined_info'][9]

'Title: Jurassic Park. Overview: A wealthy entrepreneur secretly creates a theme park featuring living dinosaurs drawn from prehistoric DNA. Before opening day, he invites a team of experts and his two eager grandchildren to experience the park and help calm anxious investors. However, the park is anything but amusing as the security systems go off-line and the dinosaurs escape. Genres: Adventure, Science Fiction. Rating: 7.39064935064935'

## Embeddings

In [7]:
# imports
import pandas as pd
import tiktoken
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

# from openai.embeddings_utils import get_embedding

from openai import OpenAI
# client = OpenAI(api_key="YOUR_API_KEY")
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], 
                                    model=model).data[0].embedding

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
md_final["n_tokens"] = md_final.combined_info.apply(lambda x: len(encoding.encode(x)))
md_final = md_final[md_final.n_tokens <= max_tokens]
len(md_final)

693

In [8]:
md_final.head()

,genres,title,overview,weighted_rate,combined_info,n_tokens
0,"[Adventure, Action, Thriller]",GoldenEye,James Bond must unmask the mysterious head of ...,6.173464,Title: GoldenEye. Overview: James Bond must un...,59
1,[Comedy],Friday,Craig and Smokey are two guys in Los Angeles h...,6.083421,Title: Friday. Overview: Craig and Smokey are ...,52
2,"[Horror, Action, Thriller, Crime]",From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,6.503176,Title: From Dusk Till Dawn. Overview: Seth Gec...,105
3,[Comedy],Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",5.109091,Title: Blue in the Face. Overview: Auggie runs...,87
4,"[Action, Adventure, Science Fiction, Family, F...",Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,5.052129,Title: Mighty Morphin Power Rangers: The Movie...,89


In [9]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

md_final["embedding"] = md_final.overview.apply(lambda x: get_embedding(x, model=embedding_model))
md_final.head()

,genres,title,overview,weighted_rate,combined_info,n_tokens,embedding
0,"[Adventure, Action, Thriller]",GoldenEye,James Bond must unmask the mysterious head of ...,6.173464,Title: GoldenEye. Overview: James Bond must un...,59,"[-0.02330085262656212, -0.01597847416996956, -..."
1,[Comedy],Friday,Craig and Smokey are two guys in Los Angeles h...,6.083421,Title: Friday. Overview: Craig and Smokey are ...,52,"[0.001594626810401678, -0.010798700153827667, ..."
2,"[Horror, Action, Thriller, Crime]",From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,6.503176,Title: From Dusk Till Dawn. Overview: Seth Gec...,105,"[-0.008669794537127018, -0.004668607376515865,..."
3,[Comedy],Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",5.109091,Title: Blue in the Face. Overview: Auggie runs...,87,"[-0.020300308242440224, -0.012248567305505276,..."
4,"[Action, Adventure, Science Fiction, Family, F...",Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,5.052129,Title: Mighty Morphin Power Rangers: The Movie...,89,"[-0.003867227118462324, -0.03921695053577423, ..."


In [10]:
md_final.rename(columns = {'embedding': 'vector'}, inplace = True)
md_final.rename(columns = {'combined_info': 'text'}, inplace = True)
md_final.to_pickle('./data/movies.pkl')

## Start working with LLMs

In [11]:
from langchain.vectorstores import LanceDB

In [12]:
import pandas as pd

md = pd.read_pickle('./data/movies.pkl')

md.head(2)

,genres,title,overview,weighted_rate,text,n_tokens,vector
0,"[Adventure, Action, Thriller]",GoldenEye,James Bond must unmask the mysterious head of ...,6.173464,Title: GoldenEye. Overview: James Bond must un...,59,"[-0.02330085262656212, -0.01597847416996956, -..."
1,[Comedy],Friday,Craig and Smokey are two guys in Los Angeles h...,6.083421,Title: Friday. Overview: Craig and Smokey are ...,52,"[0.001594626810401678, -0.010798700153827667, ..."


In [13]:
md['text'][0]

'Title: GoldenEye. Overview: James Bond must unmask the mysterious head of the Janus Syndicate and prevent the leader from utilizing the GoldenEye weapons system to inflict devastating revenge on Britain. Genres: Adventure, Action, Thriller. Rating: 6.173464373464373'

In [14]:
len(md['vector'][0])

1536

In [15]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   genres         693 non-null    object 
 1   title          693 non-null    object 
 2   overview       693 non-null    object 
 3   weighted_rate  693 non-null    float64
 4   text           693 non-null    object 
 5   n_tokens       693 non-null    int64  
 6   vector         693 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 38.0+ KB


In [ ]:
pip install lancedb

In [71]:
# # Define a schema if required
# # The schema should match the structure of your metadata
# schema = {
#     "genres": list,  # List of strings
#     "title": str,
#     "overview": str,
#     "weighted_rate": float,
#     "n_tokens": int,
#     "vector": list  # List of floats with fixed size of 1536
# }

In [16]:
import lancedb

uri = "data/sample-lancedb"
db = lancedb.connect(uri)

# table = db.create_table("movies", md)
async_db = await lancedb.connect_async(uri)

/Users/leiyu/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Synchronous client
tbl = db.create_table("my_table", data=md)

# Asynchronous client
async_tbl = await async_db.create_table("my_table2", data=md)

In [18]:
# Synchronous client
print(db.table_names())
# Asynchronous client
print(await async_db.table_names())

['my_table', 'my_table2']
['my_table', 'my_table2']


In [19]:
# Synchronous client
print(db.table_names())
# Asynchronous client
print(await async_db.table_names())

['my_table', 'my_table2']
['my_table', 'my_table2']


In [20]:
len(tbl), tbl.schema

(693,
 genres: list<item: string>
   child 0, item: string
 title: string
 overview: string
 weighted_rate: double
 text: string
 n_tokens: int64
 vector: fixed_size_list<item: float>[1536]
   child 0, item: float)

In [79]:
# md.schema

In [21]:
# Synchronous client
tbl.search(md['vector'][0]).limit(2).to_pandas()
# Asynchronous client
await async_tbl.vector_search(md['vector'][0]).limit(2).to_pandas()

,genres,title,overview,weighted_rate,text,n_tokens,vector,_distance
0,"[Adventure, Action, Thriller]",GoldenEye,James Bond must unmask the mysterious head of ...,6.173464,Title: GoldenEye. Overview: James Bond must un...,59,"[-0.023300853, -0.015978474, -0.021901755, -0....",0.000000
1,"[Adventure, Action, Thriller]",Goldfinger,Special agent 007 (Sean Connery) comes face to...,6.536484,Title: Goldfinger. Overview: Special agent 007...,89,"[-0.0130608175, -0.03419889, -0.009811793, -0....",0.227775


In [24]:
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import LanceDB
# from langchain_community.vectorstores.lancedb import LanceDB
from langchain.docstore.document import Document

from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import os

# key = os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

# Assuming md['text'] contains your raw text data
# documents = [Document(page_content=text, metadata={"source": "your_source"}) for text in md['text']]
# metadate_schema={
#     "genres": row['genres'],
#     "title": row['title'],
#     "overview": row['overview'],
#     "weighted_rate": row['weighted_rate'],
#     "n_tokens": row['n_tokens'],
#     "vector": row['vector']
# }

documents = [Document(page_content=row['text'], metadata={"source": row['weighted_rate']}) for _, row in md.iterrows()]


# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=20,
# )

# documents = text_splitter.split_documents(docs)
        
# Assuming `md` is your DataFrame and it contains columns: 'text', 'title', 'author', 'date', 'source'
# documents = [
#     Document(
#         page_content=row['text'], 
#         metadata={
#             "n_tokens": row['n_tokens'],
#             "title": row['title'],
#             "overview": row['overview'],
#             "weighted_rate": row['weighted_rate']
#         }
#     ) 
#     for _, row in md.iterrows()
# ]

# docsearch = LanceDB(connection = table, embedding = embeddings)

from lancedb.rerankers import LinearCombinationReranker

reranker = LinearCombinationReranker(weight=0.3)

docsearch = LanceDB.from_documents(documents, embeddings, reranker=reranker)
query = "What did the president say about Ketanji Brown Jackson"

In [25]:
docsearch

In [148]:
# docsearch = LanceDB(connection = tbl, embedding = embeddings)

In [26]:
query = "I'm looking for an animated action movie. What could you suggest to me?"
docs = docsearch.similarity_search(query)
docs
# docs[0].page_content

[Document(metadata={'source': '6.264471780028944'}, page_content='Title: Ice Age: Dawn of the Dinosaurs. Overview: Times are changing for Manny the moody mammoth, Sid the motor mouthed sloth and Diego the crafty saber-toothed tiger. Life heats up for our heroes when they meet some new and none-too-friendly neighbors – the mighty dinosaurs. Genres: Animation, Comedy, Family, Adventure. Rating: 6.264471780028944'),
 Document(metadata={'source': '6.831880689521596'}, page_content='Title: Despicable Me 2. Overview: Gru is recruited by the Anti-Villain League to help deal with a powerful new super criminal. Genres: Animation, Comedy, Family. Rating: 6.831880689521596'),
 Document(metadata={'source': '6.447283923466021'}, page_content='Title: Transformers. Overview: Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the D

In [27]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)

query = "I'm looking for an animated action movie. What could you suggest to me?"
result = qa({"query": query})
result['result']

/Users/leiyu/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/leiyu/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


' Kung Fu Panda 3 or Transformers would be good options for an animated action movie.'

In [28]:
result['source_documents'][2]

Document(metadata={'source': '6.447283923466021'}, page_content='Title: Transformers. Overview: Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it. Genres: Adventure, Science Fiction, Action. Rating: 6.447283923466021')

In [29]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   genres         693 non-null    object 
 1   title          693 non-null    object 
 2   overview       693 non-null    object 
 3   weighted_rate  693 non-null    float64
 4   text           693 non-null    object 
 5   n_tokens       693 non-null    int64  
 6   vector         693 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 38.0+ KB


In [30]:
df_filtered = md[md['genres'].apply(lambda x: 'Comedy' in x)]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 1 to 692
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   genres         201 non-null    object 
 1   title          201 non-null    object 
 2   overview       201 non-null    object 
 3   weighted_rate  201 non-null    float64
 4   text           201 non-null    object 
 5   n_tokens       201 non-null    int64  
 6   vector         201 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 12.6+ KB


In [31]:
df_filtered = md[md['genres'].apply(lambda x: 'Comedy' in x)]
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'data': df_filtered}), return_source_documents=True)

query = "I'm looking for a movie with animals and an adventurous plot."
result = qa({"query": query})
result

{'query': "I'm looking for a movie with animals and an adventurous plot.",
 'result': ' One Hundred and One Dalmatians or The Fox and the Hound both fit this description with their animal protagonists and adventure-filled plots.',
 'source_documents': [Document(metadata={'source': '4.978057553956835'}, page_content='Title: Cats & Dogs 2 : The Revenge of Kitty Galore. Overview: The ongoing war between the canine and feline species is put on hold when they join forces to thwart a rogue cat spy with her own sinister plans for conquest. Genres: Comedy, Family. Rating: 4.978057553956835'),
  Document(metadata={'source': '6.341426403641883'}, page_content='Title: The Fox and the Hound. Overview: When a feisty little fox named Tod is adopted into a farm family, he quickly becomes friends with a fun and adorable hound puppy named Copper. Life is full of hilarious adventures until Copper is expected to take on his role as a hunting dog -- and the object of his search is his best friend! Genres:

In [32]:
result['source_documents'][0]

Document(metadata={'source': '4.978057553956835'}, page_content='Title: Cats & Dogs 2 : The Revenge of Kitty Galore. Overview: The ongoing war between the canine and feline species is put on hold when they join forces to thwart a rogue cat spy with her own sinister plans for conquest. Genres: Comedy, Family. Rating: 4.978057553956835')

In [33]:
df_filtered = md[md['weighted_rate'].apply(lambda x: x>6)]
df_filtered['weighted_rate'].describe()

count    236.000000
mean       6.741597
std        0.501437
min        6.000384
25%        6.308121
50%        6.651140
75%        7.114158
max        8.264788
Name: weighted_rate, dtype: float64

In [34]:
# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", 
#     retriever=docsearch.as_retriever(search_kwargs={'filter': {'adult':'False'}}), return_source_documents=True)
df_filtered = md[md['weighted_rate'].apply(lambda x: x>6)]
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'data': df_filtered}), return_source_documents=True)


query = "I'm looking for a movie with animals and an adventurous plot."
result = qa({"query": query})
result

{'query': "I'm looking for a movie with animals and an adventurous plot.",
 'result': ' The Fox and the Hound or One Hundred and One Dalmatians could fit this description. They both have animals as main characters and involve adventure and danger.',
 'source_documents': [Document(metadata={'source': '4.978057553956835'}, page_content='Title: Cats & Dogs 2 : The Revenge of Kitty Galore. Overview: The ongoing war between the canine and feline species is put on hold when they join forces to thwart a rogue cat spy with her own sinister plans for conquest. Genres: Comedy, Family. Rating: 4.978057553956835'),
  Document(metadata={'source': '6.341426403641883'}, page_content='Title: The Fox and the Hound. Overview: When a feisty little fox named Tod is adopted into a farm family, he quickly becomes friends with a fun and adorable hound puppy named Copper. Life is full of hilarious adventures until Copper is expected to take on his role as a hunting dog -- and the object of his search is his b

In [35]:
result['source_documents'][0]

Document(metadata={'source': '4.978057553956835'}, page_content='Title: Cats & Dogs 2 : The Revenge of Kitty Galore. Overview: The ongoing war between the canine and feline species is put on hold when they join forces to thwart a rogue cat spy with her own sinister plans for conquest. Genres: Comedy, Family. Rating: 4.978057553956835')

In [ ]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'filter': {weighted_rate__gt:7}}), return_source_documents=True)
query = "I'm looking for a movie with animals and an adventurous plot."
result = qa({"query": query})
result

In [36]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)
retriever = docsearch.as_retriever(return_source_documents = True)
tool = create_retriever_tool(
    retriever,
    "movies",
    "Searches and returns recommendations about movies."
)
tools = [tool]
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)
result = agent_executor({"input": "suggest me some action movies"})

/Users/leiyu/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `movies` with `{'query': 'action'}`


Title: In the Name of the King: A Dungeon Siege Tale. Overview: A man named Farmer sets out to rescue his kidnapped wife and avenge the death of his son -- two acts committed by the Krugs, a race of animal-warriors who are controlled by the evil Gallian. Genres: Adventure, Fantasy, Action, Drama. Rating: 4.690922844175492

Title: Hard Target 2. Overview: Forced into a deadly cat-and-mouse game, a disgraced mixed martial arts fighter is hunted through the jungles of Southeast Asia. Genres: Action, Thriller. Rating: 5.055230125523012

Title: [REC]². Overview: The action continues from [REC], with the medical officer and a SWAT team outfitted with video cameras are sent into the sealed off apartment to control the situation. Genres: Thriller, Horror. Rating: 5.800000000000001

Title: RED 2. Overview: Retired C.I.A. agent Frank Moses reunites his unlikely team of elite operatives for a global quest to 

In [37]:
print(qa.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [38]:
from langchain.prompts import PromptTemplate
template = """You are a movie recommender system that help users to find movies that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three movies, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Your response:"""
 
PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

In [39]:
PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)
query = "I'm looking for a funny action movie, any suggestion?"
result = qa({'query':query})
print(result['result'])


1. Deadpool: A former Special Forces operative turned mercenary is subjected to a rogue experiment that leaves him with accelerated healing powers, adopting the alter ego Deadpool. With its witty humor and non-stop action, this movie is perfect for anyone looking for a funny action movie.

2. Hot Fuzz: A top London cop is transferred to a small town with a dark secret. As he delves deeper into the town's mysteries, he must team up with a bumbling local constable to uncover the truth. This movie is a perfect blend of action and comedy, with hilarious one-liners and over-the-top action sequences.

3. The Nice Guys: In 1970s Los Angeles, two mismatched private detectives are hired to investigate the disappearance of a young woman and the death of a porn star. As they dig deeper, they uncover a conspiracy that puts their lives in danger. With its clever dialogue and thrilling action scenes, this movie is sure to satisfy anyone looking for a funny action movie.


In [40]:
from langchain.prompts import PromptTemplate
template_prefix = """You are a movie recommender system that help users to find movies that match their preferences.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}"""
user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""
template_suffix= """Question: {question}
Your response:"""
user_info = user_info.format(age = 18, gender = 'female')
COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find movies that match their preferences.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: female
Question: {question}
Your response:


In [41]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)
result = qa({'query':query})
result['result']

'Based on your preferences, I would recommend RED 2. It has a mix of action, comedy, and thriller genres, which aligns with your interest in action and humor. The rating of 6.0934409687184665 suggests that it is a well-received movie and worth checking out. Other options could be The Expendables 2 or Bad Ass 2: Bad Asses, both of which also have a mix of action and humor, but with slightly lower ratings.'

In [42]:
import pandas as pd
data = {
    "username": ["Alice", "Bob"],
    "age": [25, 32],
    "gender": ["F", "M"],
    "movies": [
        [("Transformers: The Last Knight", 7), ("Pokémon: Spell of the Unknown", 5)],
        [("Bon Cop Bad Cop 2", 8), ("Goon: Last of the Enforcers", 9)]
    ]
}
# Convert the "movies" column into dictionaries
for i, row_movies in enumerate(data["movies"]):
    movie_dict = {}
    for movie, rating in row_movies:
        movie_dict[movie] = rating
    data["movies"][i] = movie_dict
# Create a pandas DataFrame
df = pd.DataFrame(data)
df.head()

,username,age,gender,movies
0,Alice,25,F,"{'Transformers: The Last Knight': 7, 'Pokémon:..."
1,Bob,32,M,"{'Bon Cop Bad Cop 2': 8, 'Goon: Last of the En..."


In [43]:
template_prefix = """You are a movie recommender system that help users to find movies that match their preferences.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}"""
user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}
Movies already seen alongside with rating: {movies}"""
template_suffix= """Question: {question}
Your response:"""

In [44]:
age = df.loc[df['username']=='Alice']['age'][0]
gender = df.loc[df['username']=='Alice']['gender'][0]
movies = ''
# Iterate over the dictionary and output movie name and rating
for movie, rating in df['movies'][0].items():
    output_string = f"Movie: {movie}, Rating: {rating}" + "\n"
    movies+=output_string
    #print(output_string)
user_info = user_info.format(age = age, gender = gender, movies = movies)
COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find movies that match their preferences.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 25
Gender: F
Movies already seen alongside with rating: Movie: Transformers: The Last Knight, Rating: 7
Movie: Pokémon: Spell of the Unknown, Rating: 5

Question: {question}
Your response:


In [45]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)
query = "Can you suggest me some action movie based on my background?"
result = qa({'query':query})
result['result']

' I would recommend checking out The Raid 2 and Rambo based on your previous ratings and the fact that you enjoy action movies. Both have high ratings and are in the same genre as the movies you have already seen. '

In [131]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

llm = ChatOpenAI(temperature = 0)

system_message = SystemMessage(
        content=(
            "Do your best to answer the questions. "
            "if there are more than one argument for the single-input tool, reason step by step and treat them as single input. "
            "relevant information, only if neccessary"
        )
)

# This is needed for both the memory and the prompt
memory_key = "history"

memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )
agent_executor = create_conversational_retrieval_agent(llm=llm, tools=tools, prompt = prompt, verbose=True)

result = agent_executor({"input": "I liked a lot kung fu panda 1 and 2. Could you suggest me some similar movies?"})
result



> Entering new AgentExecutor chain...

Invoking: `movies` with `{'query': 'kung fu panda'}`


Title: Kung Fu Panda. Overview: When the Valley of Peace is threatened, lazy Po the panda discovers his destiny as the "chosen one" and trains to become a kung fu hero, but transforming the unsleek slacker into a brave warrior won't be easy. It's up to Master Shifu and the Furious Five -- Tigress, Crane, Mantis, Viper and Monkey -- to give it a try. Genres: Adventure, Animation, Family, Comedy. Rating: 6.675006821282402

Title: Kung Fu Panda 2. Overview: Po is now living his dream as The Dragon Warrior, protecting the Valley of Peace alongside his friends and fellow kung fu masters, The Furious Five - Tigress, Crane, Mantis, Viper and Monkey. But Po’s new life of awesomeness is threatened by the emergence of a formidable villain, who plans to use a secret, unstoppable weapon to conquer China and destroy kung fu. It is up to Po and The Furious Five to journey across China to face this threat 

{'input': 'I liked a lot kung fu panda 1 and 2. Could you suggest me some similar movies?',
 'chat_history': [HumanMessage(content='I liked a lot kung fu panda 1 and 2. Could you suggest me some similar movies?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"kung fu panda"}', 'name': 'movies'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-541b7725-e039-461e-a678-a7893ee42273-0'),
  FunctionMessage(content='Title: Kung Fu Panda. Overview: When the Valley of Peace is threatened, lazy Po the panda discovers his destiny as the "chosen one" and trains to become a kung fu hero, but transforming the unsleek slacker into a brave warrior won\'t be easy. It\'s up to Master Shifu and the Furious Five -- Tigress, Crane, Mantis, Viper and Monkey -- to give it a try. Genres: Adventur

## Prompt engineering

In [132]:
from langchain.prompts import PromptTemplate

template = """You are a movie recommender system that help users to find movies that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, suggest three movies, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "I'm looking for a funny action movie, any suggestion?"
result = qa({'query':query})
print(result['result'])



1. Deadpool - A wisecracking mercenary gets enhanced with superhuman powers and sets out on a mission to track down the man who nearly destroyed his life. With a mix of action and comedy, this movie is perfect for those looking for a humorous twist on the traditional action genre.
2. 21 Jump Street - Two underachieving cops are sent back to a local high school to blend in and bring down a synthetic drug ring. With a great blend of action and comedy, this movie is sure to keep you entertained and laughing.
3. Hot Fuzz - A top London cop is assigned to a seemingly dull town in the countryside, but his new partner uncovers a series of grisly murders. This movie is a hilarious parody of the action genre, with plenty of over-the-top action scenes and witty humor.


In [133]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that help users to find movies that match their preferences. 
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'female')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)


You are a movie recommender system that help users to find movies that match their preferences. 
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: female
Question: {question}
Your response:


In [134]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "Can you suggest me some action movie?"
result = qa({'query':query})
result['result']


' Based on your age and gender, I would recommend The Raid 2. It has elements of action, crime, and thriller, which may appeal to your preferences. It also has a higher rating compared to the other options, indicating that it may be a well-received film by viewers.'

In [135]:
result['source_documents']

[Document(metadata={'source': 'your_source'}, page_content='Title: Hard Target 2. Overview: Forced into a deadly cat-and-mouse game, a disgraced mixed martial arts fighter is hunted through the jungles of Southeast Asia. Genres: Action, Thriller. Rating: 5.055230125523012'),
 Document(metadata={'source': 'your_source'}, page_content='Title: The Raid 2. Overview: After fighting his way through an apartment building populated by an army of dangerous criminals and escaping with his life, SWAT team member Rama goes undercover, joining a powerful Indonesian crime syndicate to protect his family and uncover corrupt members of his own force. Genres: Action, Crime, Thriller. Rating: 6.7086887835703'),
 Document(metadata={'source': 'your_source'}, page_content='Title: RED 2. Overview: Retired C.I.A. agent Frank Moses reunites his unlikely team of elite operatives for a global quest to track down a missing portable nuclear device. Genres: Action, Comedy, Crime, Thriller. Rating: 6.09344096871846

## Content based

In [136]:
import pandas as pd

data = {
    "username": ["Alice", "Bob"],
    "age": [25, 32],
    "gender": ["F", "M"],
    "movies": [
        [("Transformers: The Last Knight", 7), ("Pokémon: Spell of the Unknown", 5)],
        [("Bon Cop Bad Cop 2", 8), ("Goon: Last of the Enforcers", 9)]
    ]
}

# Convert the "movies" column into dictionaries
for i, row_movies in enumerate(data["movies"]):
    movie_dict = {}
    for movie, rating in row_movies:
        movie_dict[movie] = rating
    data["movies"][i] = movie_dict

# Create a pandas DataFrame
df = pd.DataFrame(data)

df.head()


,username,age,gender,movies
0,Alice,25,F,"{'Transformers: The Last Knight': 7, 'Pokémon:..."
1,Bob,32,M,"{'Bon Cop Bad Cop 2': 8, 'Goon: Last of the En..."


In [137]:
template_prefix = """You are a movie recommender system that help users to find movies that match their preferences. 
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}
Movies already seen alongside with rating: {movies}"""

template_suffix= """Question: {question}
Your response:"""

In [138]:
age = df.loc[df['username']=='Alice']['age'][0]
gender = df.loc[df['username']=='Alice']['gender'][0]

movies = ''
# Iterate over the dictionary and output movie name and rating
for movie, rating in df['movies'][0].items():
    output_string = f"Movie: {movie}, Rating: {rating}" + "\n"
    movies+=output_string
    #print(output_string)
user_info = user_info.format(age = age, gender = gender, movies = movies)

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find movies that match their preferences. 
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 25
Gender: F
Movies already seen alongside with rating: Movie: Transformers: The Last Knight, Rating: 7
Movie: Pokémon: Spell of the Unknown, Rating: 5

Question: {question}
Your response:


In [139]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "Can you suggest me some action movie based on my background?"
result = qa({'query':query})
result['result']

' Based on your age, gender, and movies already seen and rated, I would recommend the following action movies for you: \n1. Mad Max: Fury Road - Rating: 8.1\n2. Atomic Blonde - Rating: 6.7\n3. Hanna - Rating: 6.8\n4. Fast Five - Rating: 7.3\n5. Lucy - Rating: 6.4\n\nAll of these movies have a similar genre and rating to the ones you have already seen, so I believe you will enjoy them as well. Happy watching!'

In [140]:
result['source_documents']

[Document(metadata={'source': 'your_source'}, page_content='Title: Hard Target 2. Overview: Forced into a deadly cat-and-mouse game, a disgraced mixed martial arts fighter is hunted through the jungles of Southeast Asia. Genres: Action, Thriller. Rating: 5.055230125523012'),
 Document(metadata={'source': 'your_source'}, page_content='Title: The Raid 2. Overview: After fighting his way through an apartment building populated by an army of dangerous criminals and escaping with his life, SWAT team member Rama goes undercover, joining a powerful Indonesian crime syndicate to protect his family and uncover corrupt members of his own force. Genres: Action, Crime, Thriller. Rating: 6.7086887835703'),
 Document(metadata={'source': 'your_source'}, page_content='Title: Ong Bak 2. Overview: Moments from death a young man is rescued by a renowned warrior. Realizing unsurpassed physical potential in the young boy he trains him into the most dangerous man alive. As he becomes a young man he goes on 

In [141]:
import streamlit as st
st.set_page_config(page_title="GlobeBotter", page_icon="")
st.header(' Welcome to MovieHarbor, your favourite movie recommender')

2024-08-17 16:42:55.741 
  command:

    streamlit run /Users/leiyu/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()